In [11]:
import openmc
import numpy as np
import openmc.data
from pathlib import Path
import matplotlib.pyplot as plt


import os

os.environ["OPENMC_CROSS_SECTIONS"] = "/home/harriet/OPENMC/mcnp_endfb70/cross_sections.xml"


###############################################################################
#                      Simulation Input File Parameters
###############################################################################

# OpenMC simulation parameters
batches = 30
inactive = 10
particles = 100

statepoints_folder = Path('statepoints_folder')

###############################################################################
#                 Exporting to OpenMC materials.xml file
###############################################################################

# Create materials
rock = openmc.Material(material_id=1, name='Rock')
rock.add_element('Si', 0.0085514332)
rock.add_nuclide('O16', 0.622)
rock.add_element('Al', 0.101)
rock.add_nuclide('Fe56',1.0)

rock.set_density('g/cm3', 2.65)


water = openmc.Material(material_id=2,name='Water')
water.add_element('H', 2)
water.add_nuclide('O16', 1)
water.set_density('g/cm3', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')

HDPE_material = openmc.Material(material_id=3,name='HDPE_Material')
HDPE_material.add_nuclide('C12', 1.977844)
HDPE_material.add_element('H', 4)
HDPE_material.set_density('g/cm3', 0.93)
# Instantiate a Materials collection and export to XML
materials_file = openmc.Materials([water, rock,HDPE_material])
materials_file.export_to_xml()


###############################################################################
#                 Exporting to OpenMC geometry.xml file
###############################################################################


# Create surfaces
wall_thickness = 0.5
water_thickness = 0.5
wall_length= 1.5
water_length= 1.5
HDPE_material_length =2.5



# Create surfaces for rock wall
left_surface_rock = openmc.XPlane(surface_id=1,x0=-wall_length/2, boundary_type='vacuum')
right_surface_rock = openmc.XPlane(surface_id=2,x0=wall_length/2, boundary_type='vacuum')
bottom_surface_rock = openmc.YPlane(surface_id=3,y0=-wall_length/2, boundary_type='vacuum')
top_surface_rock = openmc.YPlane(surface_id=4,y0=wall_length/2, boundary_type='vacuum')
front_surface_rock = openmc.ZPlane(surface_id=5,z0=-wall_thickness/2, boundary_type='vacuum')
back_surface_rock = openmc.ZPlane(surface_id=6,z0=wall_thickness/2, boundary_type='vacuum')

# Create surfaces for water wall
left_surface_water = openmc.XPlane(surface_id=7,x0=-water_length/2, boundary_type='vacuum')
right_surface_water = openmc.XPlane(surface_id=8,x0=water_length/2, boundary_type='vacuum')
bottom_surface_water = openmc.YPlane(surface_id=9,y0=-water_length/2, boundary_type='vacuum')
top_surface_water = openmc.YPlane(surface_id=10,y0=water_length/2, boundary_type='vacuum')
front_surface_water = openmc.ZPlane(surface_id=11,z0=-water_thickness/2, boundary_type='vacuum')
back_surface_water = openmc.ZPlane(surface_id=12,z0=water_thickness/2, boundary_type='vacuum')

# Create surfaces for HDPE wall
left_surface_HDPE = openmc.XPlane(surface_id=13,x0=-HDPE_material_length/2, boundary_type='vacuum')
right_surface_HDPE = openmc.XPlane(surface_id=14,x0=HDPE_material_length/2, boundary_type='vacuum')
bottom_surface_HDPE = openmc.YPlane(surface_id=15,y0=-HDPE_material_length/2, boundary_type='vacuum')
top_surface_HDPE = openmc.YPlane(surface_id=16,y0=HDPE_material_length/2, boundary_type='vacuum')
front_surface_HDPE = openmc.ZPlane(surface_id=17,z0=-water_thickness/2, boundary_type='vacuum')
back_surface_HDPE = openmc.ZPlane(surface_id=18,z0=water_thickness/2, boundary_type='vacuum')

# Instantiate Cells_for rock wall and water wall. 
cell1 = openmc.Cell(cell_id=1, name='Cell 1')
cell2 = openmc.Cell(cell_id=2, name='Cell 2')
cell3 = openmc.Cell(cell_id=3, name='Cell 3')

cell1.region = +left_surface_rock & -right_surface_rock & +bottom_surface_rock & -top_surface_rock & +front_surface_rock & -back_surface_rock 
cell2.region = +left_surface_water & -right_surface_water & +bottom_surface_water & -top_surface_water & +front_surface_water & -back_surface_water 
cell3.region = +left_surface_HDPE & -right_surface_HDPE & +bottom_surface_HDPE & -top_surface_HDPE & +front_surface_HDPE & -back_surface_HDPE
# Register Materials with Cells
cell1.fill = rock
cell2.fill = water
cell3.fill = HDPE_material
# Create universe
root = openmc.Universe(universe_id=0, name='root universe')
root.add_cell(cell1)
root.add_cell(cell2)
root.add_cell(cell3)

# Instantiate a Geometry, register the root Universe, and export to XML
geometry = openmc.Geometry(root)
geometry.export_to_xml()




###############################################################################
#                   Exporting to OpenMC settings.xml file
###############################################################################

# Create settings
settings = openmc.Settings()
settings.run_mode = 'fixed source'
settings.particles = particles
settings.batches = batches
settings.inactive = inactive
# Define energy distribution
energy = openmc.stats.Discrete([14e6], [1.0])  # Single energy of 14 MeV


source = openmc.IndependentSource(space=openmc.stats.Point(), energy=energy)
source.particle="neutron"
settings.source = source

# Export to "settings.xml"
settings.export_to_xml()

###############################################################################
#                   Exporting to OpenMC tallies.xml file
###############################################################################
# Instantiate an empty Tallies object
tallies = openmc.Tallies()

# Create mesh which will be used for tally
mesh = openmc.RegularMesh(mesh_id=1)
mesh.dimension = [100, 100,10 ]
mesh.lower_left,mesh.upper_right = geometry.bounding_box



# Create mesh filter for tally
mesh_filter = openmc.MeshFilter(mesh)
# Create mesh tally to score flux and fission rate
tally = openmc.Tally(tally_id=1,name='flux')
tally.filters = [mesh_filter]
tally.scores = ['flux', 'fission']
tallies.append(tally)
# Export to "tallies.xml"
tallies.export_to_xml()


# deletes old statepoint and summary files
!rm s*.h5
# Create a simulation model
model = openmc.Model(geometry, materials_file,settings,  tallies)

# Run the simulation
output = model.run(tracks=True)

###############################################################################
#                   Exporting to OpenMC plots.xml file
###############################################################################

plot = openmc.Plot(plot_id=1)
plot.origin = [0, 0, 0]
plot.width = [4, 4]
plot.pixels = [400, 400]
plot.color_by = 'material'

# Instantiate a Plots collection and export to XML
plot_file = openmc.Plots([plot])
plot.geometry=geometry
openmc.plot_inline(plot)
plot_file.export_to_xml()


# Load the statepoint file
sp = openmc.StatePoint('/home/harriet/OPENMC/openmc/examples/my_work/statepoint.30.h5')
tally = sp.get_tally(scores=['flux'])
tracks = openmc.Tracks('tracks.h5')
pp = tracks.filter(particle='neutron')
pp.plot()



'''
vox_plot = openmc.Plot(plot_id=2)
vox_plot.type = 'voxel'
vox_plot.width = [100., 100.,50.]
vox_plot.pixels = [400, 400,200]
vox_plot.color_by = 'material'

# Instantiate a Plots collection and export to XML
vox_plot = openmc.Plots([vox_plot])
vox_plot.geometry=geometry
openmc.vox_plot(vox_plot)

'''
'''


# Extract results
#results_transmitted = output.get_tally(name='transmitted')

# Plot the geometry
openmc.plot_geometry(geom)
#openmc.plot_geometry(geom, filename='geometry_plot.png')
plt.show()


# Print results
#print("Number of neutrons transmitted through water:", results_transmitted.mean)
'''


rm: cannot remove 's*.h5': No such file or directory


/home/harriet/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/harriet/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/harriet/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/harriet/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=1.
  warn(msg, IDWarning)
/home/harriet/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=2.
  warn(msg, IDWarning)
/home/harriet/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance al

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

RuntimeError: Could not find nuclide C12 in the nuclear data library. -------------------------------------------------------------------------- MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD with errorcode -1. NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes. You may or may not see output from other processes, depending on exactly when Open MPI kills them. --------------------------------------------------------------------------